In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score,StratifiedKFold , GridSearchCV,train_test_split
from sklearn.metrics import make_scorer
from lightgbm import LGBMClassifier
from lightgbm.callback import early_stopping  # ✅ Import 추가
import optuna

In [15]:
train = pd.read_csv('train.csv').drop(columns=['ID'])
test = pd.read_csv('test.csv').drop(columns=['ID'])

In [16]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']
X = X.drop(columns=['임신 시도 또는 마지막 임신 경과 연수', '착상 전 유전 검사 사용 여부', 'PGD 시술 여부','PGS 시술 여부','난자 해동 경과일','배아 해동 경과일',"난자 채취 경과일"])
test = test.drop(columns=['임신 시도 또는 마지막 임신 경과 연수', '착상 전 유전 검사 사용 여부', 'PGD 시술 여부','PGS 시술 여부','난자 해동 경과일','배아 해동 경과일',"난자 채취 경과일"])
#임신 시도 또는 마지막 임신 경과 연수 열 삭제
#착상 전 유전 검사 사용 여부 삭제
#PGD 시술 여부 삭제
#PGS 시술 여부 삭제
#난자 해동 경과일 삭제
#배아 해동 경과일 삭제



In [17]:
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부"
]

# One-Hot Encoder 초기화
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# 인코딩 적용
X_train_encoded = onehot_encoder.fit_transform(X[categorical_columns])
X_test_encoded = onehot_encoder.transform(test[categorical_columns])

# 인코딩된 데이터를 데이터프레임으로 변환
X_train_encoded = pd.DataFrame(X_train_encoded, columns=onehot_encoder.get_feature_names_out(categorical_columns))
X_test_encoded = pd.DataFrame(X_test_encoded, columns=onehot_encoder.get_feature_names_out(categorical_columns))

# 기존 데이터프레임과 결합
X_train_encoded = pd.concat([X.drop(columns=categorical_columns).reset_index(drop=True), X_train_encoded], axis=1)
X_test_encoded = pd.concat([test.drop(columns=categorical_columns).reset_index(drop=True), X_test_encoded], axis=1)

X_train_encoded.columns = X_train_encoded.columns.str.replace(r'[\[\]{}":,]', '', regex=True)
X_test_encoded.columns = X_test_encoded.columns.str.replace(r'[\[\]{}":,]', '', regex=True)
X_train_encoded.columns = X_train_encoded.columns.str.replace(" ", "_")
X_test_encoded.columns = X_test_encoded.columns.str.replace(" ", "_")

In [18]:
numeric_columns = [
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 혼합 경과일",
    "배아 이식 경과일",
]

#인코딩할 때 결측치도 그냥 인코딩해줌
X_train_encoded[numeric_columns] = X_train_encoded[numeric_columns].fillna(0)
X_test_encoded[numeric_columns] = X_test_encoded[numeric_columns].fillna(0)
y=y.fillna(0)

KeyError: "None of [Index(['총 생성 배아 수', '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '이식된 배아 수',\n       '미세주입 배아 이식 수', '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수',\n       '수집된 신선 난자 수', '저장된 신선 난자 수', '혼합된 난자 수', '파트너 정자와 혼합된 난자 수',\n       '기증자 정자와 혼합된 난자 수', '난자 혼합 경과일', '배아 이식 경과일'],\n      dtype='object')] are in the [columns]"

In [10]:
import optuna
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_train_encoded, y, test_size=0.2, stratify=y, random_state=42)

# Stratified K-Fold 설정
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=101)

# Optuna 최적화 함수 정의
def objective(trial):
    params = {
        'objective': 'binary',
        'metric': 'auc',
        'num_leaves': trial.suggest_int('num_leaves', 10, 100),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'n_estimators': 100,  # 최대한 크게 설정하고 early_stopping으로 조기 종료
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0)
    }

    roc_auc_scores = []

    # K-Fold 교차검증 수행
    for train_idx, valid_idx in skf.split(X_train, y_train):
        # DataFrame이므로 .iloc을 사용
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[valid_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[valid_idx]

        # 모델 생성
        model = LGBMClassifier(class_weight="balanced", random_state=101, **params)

        # 조기 종료(Early Stopping) 적용
        model.fit(
            X_tr, y_tr,
            eval_set=[(X_val, y_val)],
            eval_metric="auc",
            callbacks=[early_stopping(20)]
        )

        # 검증 AUC 계산
        y_val_pred = model.predict_proba(X_val)[:, 1]
        roc_auc_scores.append(roc_auc_score(y_val, y_val_pred))

    return np.mean(roc_auc_scores)  # 평균 ROC-AUC 반환

# Optuna 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# 최적의 하이퍼파라미터 출력
print("최적의 하이퍼파라미터:", study.best_params)

# 최적의 모델 생성 및 평가
best_params = study.best_params
best_model = LGBMClassifier(class_weight="balanced", random_state=101, **best_params)
best_model.fit(X_train, y_train)

y_pred_proba = best_model.predict_proba(X_test)[:, 1]

# 최종 ROC-AUC 점수 출력
final_auc = roc_auc_score(y_test, y_pred_proba)
print("테스트 데이터 ROC-AUC:", final_auc)


ValueError: Input y contains NaN.

In [11]:
model = LGBMClassifier(class_weight="balanced", random_state=101,**best_params)
model.fit(X_train_encoded, y)
# Stratified K-Fold 설정 (클래스 비율 유지)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=101)

# 5-Fold 교차 검증 수행 (ROC-AUC 기준)
roc_auc_scores = cross_val_score(model, X_train_encoded, y, cv=skf, scoring="roc_auc")

# 결과 출력
print("각 Fold의 ROC-AUC:", roc_auc_scores)
print("평균 ROC-AUC:", roc_auc_scores.mean())

NameError: name 'best_params' is not defined

In [6]:
pred_proba = best_model.predict_proba(X_test_encoded)[:, 1]

sample_submission = pd.read_csv('sample_submission.csv')
sample_submission['probability'] = pred_proba

sample_submission.to_csv('baseline_submit.csv', index=False)

NameError: name 'best_model' is not defined